<a href="https://colab.research.google.com/github/econdatatech/AIML429/blob/main/PGM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pgmpy
from itertools import combinations
import pandas
import networkx as nx
from sklearn.metrics import f1_score
import numpy as np
from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch,K2Score, BicScore, BDeuScore,MmhcEstimator,BDsScore

import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.7 MB/s eta 0:00:00


In [3]:
samples=pandas.DataFrame(pandas.read_csv("https://raw.githubusercontent.com/econdatatech/AIML429/main/pet-prey-temp.txt", sep='\t', lineterminator='\r',converters={'pet':lambda x:x.replace('\n','')})[:-1])

In [4]:
samples

,pet,prey,temp
0,dog,skink,cold
1,dog,mouse,warm
2,dog,mouse,warm
3,dog,skink,warm
4,dog,skink,cold
...,...,...,...
995,dog,sparrow,cold
996,dog,mouse,warm
997,dog,skink,cold
998,dog,skink,cold


In [5]:
# What 3-node belief net structure(s) would best match the dependencies exhibited by this data? (Justify your answer,
#i.e. are there independencies? conditional or otherwise? Does observation of one induce
#dependencies among the others?)

es = ExhaustiveSearch(samples,scoring_method=K2Score(samples))
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())


es = ExhaustiveSearch(samples,scoring_method=BicScore(samples))
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

[('pet', 'prey'), ('pet', 'temp')]

All DAGs by score:
-2475.3085773118005 [('pet', 'prey'), ('pet', 'temp')]
-2475.3379855966637 [('pet', 'prey'), ('temp', 'pet')]
-2475.4812679125957 [('pet', 'prey')]
-2475.5316868232476 [('pet', 'temp'), ('prey', 'pet')]
-2475.704377424043 [('prey', 'pet')]
-2477.7280244022786 [('prey', 'pet'), ('temp', 'pet')]
-2477.930560367481 [('pet', 'temp')]
-2477.9599686523443 [('temp', 'pet')]
-2478.1032509682764 []
-2479.7608693619272 [('pet', 'prey'), ('prey', 'temp')]
-2479.819566924289 [('prey', 'pet'), ('temp', 'prey')]
-2479.9839788733743 [('prey', 'temp'), ('prey', 'pet')]
-2481.355126466882 [('pet', 'prey'), ('pet', 'temp'), ('temp', 'prey')]
-2481.3845347517454 [('pet', 'prey'), ('temp', 'pet'), ('temp', 'prey')]
-2481.5278170676775 [('pet', 'prey'), ('temp', 'prey')]
-2481.6582921762706 [('pet', 'prey'), ('pet', 'temp'), ('prey', 'temp')]
-2481.843213902525 [('prey', 'pet'), ('temp', 'pet'), ('temp', 'prey')]
-2481.8814016877177 [('pet', 'temp'), (

In [ ]:
data=samples
mmhc = MmhcEstimator(data)
skeleton = mmhc.mmpc()
print("Part 1) Skeleton: ", skeleton.edges())

# use hill climb search to orient the edges:
hc = HillClimbSearch(data)
model = hc.estimate(tabu_length=10, white_list=skeleton.to_directed().edges(), scoring_method=BDeuScore(data))
print("Part 2) Model:    ", model.edges())

Part 1) Skeleton:  [('pet', 'prey')]


  0%|          | 0/1000000 [00:00<?, ?it/s]

Part 2) Model:     [('prey', 'pet')]


In [ ]:
est = PC(data)
skel, seperating_sets = est.build_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = pdag.to_dag()
print("DAG edges:        ", model.edges())

NameError: name 'PC' is not defined